In [2]:
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.datasets import ImageFolder



In [1]:
device = 'cuda'

In [3]:
train_path = "data/pizza_steak_sushi/train"
test_path = "data/pizza_steak_sushi/test"

In [4]:
data_tranform = transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

In [7]:
train_data = ImageFolder(root=train_path,transform=data_tranform,target_transform=None)
test_data = ImageFolder(root=test_path,transform=data_tranform)

In [10]:
train_dataloader = DataLoader(dataset=train_data,batch_size=32,shuffle=True)
test_dataloader = DataLoader(dataset=test_data,batch_size=32,shuffle=False)

In [16]:
images, labels = next(iter(train_dataloader))
images.shape

torch.Size([32, 3, 64, 64])

In [8]:
from torch import nn

In [45]:
class TinyVGG(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(in_channels=3,out_channels=25,kernel_size=3,padding=1,stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=25,out_channels=25,kernel_size=3,padding=1,stride=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(in_channels=25,out_channels=25,kernel_size=3,padding=1,stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=25,out_channels=25,kernel_size=3,padding=1,stride=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        self.linear = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=16*16*25,out_features=3)
        )
    def forward(self,x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.linear(x)
        return x

In [46]:
model = TinyVGG().to(device=device)

In [19]:
from torchinfo import summary


In [47]:
summary(model,input_size=(32,3,64,64))

Layer (type:depth-idx)                   Output Shape              Param #
TinyVGG                                  [32, 3]                   --
├─Sequential: 1-1                        [32, 25, 32, 32]          --
│    └─Conv2d: 2-1                       [32, 25, 64, 64]          700
│    └─ReLU: 2-2                         [32, 25, 64, 64]          --
│    └─Conv2d: 2-3                       [32, 25, 64, 64]          5,650
│    └─ReLU: 2-4                         [32, 25, 64, 64]          --
│    └─MaxPool2d: 2-5                    [32, 25, 32, 32]          --
├─Sequential: 1-2                        [32, 25, 16, 16]          --
│    └─Conv2d: 2-6                       [32, 25, 32, 32]          5,650
│    └─ReLU: 2-7                         [32, 25, 32, 32]          --
│    └─Conv2d: 2-8                       [32, 25, 32, 32]          5,650
│    └─ReLU: 2-9                         [32, 25, 32, 32]          --
│    └─MaxPool2d: 2-10                   [32, 25, 16, 16]          --
├─Seq

In [31]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(),lr=0.01)

In [48]:


epochs = 20
for epoch in range(epochs):
    train_loss = 0
    train_acc = 0
    test_loss = 0
    test_acc = 0
    model.train()
    for x,y in train_dataloader:
        x,y = x.to(device),y.to(device)

        y_logits = model(x)
        y_pred = torch.softmax(y_logits,dim=1).argmax(dim=1)
        loss = loss_fn(y_logits,y)
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        acc = (y_pred == y).sum().item() / len(y)
        train_acc += acc
    train_loss /= len(train_dataloader)
    train_acc /= len(train_dataloader)
    model.eval()
    with torch.inference_mode():  
        for x,y in test_dataloader:
            x,y = x.to(device),y.to(device)
            y_logits = model(x)
            y_pred = torch.softmax(y_logits,dim=1).argmax(dim=1)
            loss = loss_fn(y_logits,y)
            test_loss += loss.item()
            acc = (y_pred == y).sum().item() / len(y)
            test_acc += acc
        test_loss /= len(test_dataloader)
        test_acc /= len(test_dataloader)
    print(f"Epoch {epoch+1}/{epochs} | "
          f"Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | "
          f"Test Loss: {test_loss:.4f} Acc: {test_acc:.4f}")







Epoch 1/20 | Train Loss: 1.0983 Acc: 0.3320 | Test Loss: 1.0979 Acc: 0.2282
Epoch 2/20 | Train Loss: 1.0986 Acc: 0.3164 | Test Loss: 1.0977 Acc: 0.2282
Epoch 3/20 | Train Loss: 1.0987 Acc: 0.3086 | Test Loss: 1.0977 Acc: 0.2386
Epoch 4/20 | Train Loss: 1.0974 Acc: 0.4414 | Test Loss: 1.0980 Acc: 0.2386
Epoch 5/20 | Train Loss: 1.0976 Acc: 0.4414 | Test Loss: 1.0980 Acc: 0.2491
Epoch 6/20 | Train Loss: 1.0975 Acc: 0.4336 | Test Loss: 1.0979 Acc: 0.2491
Epoch 7/20 | Train Loss: 1.1008 Acc: 0.3281 | Test Loss: 1.0978 Acc: 0.2282
Epoch 8/20 | Train Loss: 1.0977 Acc: 0.4375 | Test Loss: 1.0979 Acc: 0.2282
Epoch 9/20 | Train Loss: 1.0982 Acc: 0.3125 | Test Loss: 1.0977 Acc: 0.2386
Epoch 10/20 | Train Loss: 1.0985 Acc: 0.3281 | Test Loss: 1.0980 Acc: 0.2282
Epoch 11/20 | Train Loss: 1.0973 Acc: 0.4414 | Test Loss: 1.0981 Acc: 0.2282
Epoch 12/20 | Train Loss: 1.0971 Acc: 0.4336 | Test Loss: 1.0980 Acc: 0.2386
Epoch 13/20 | Train Loss: 1.0980 Acc: 0.4297 | Test Loss: 1.0979 Acc: 0.2386
Epoch 14

In [49]:
model_path = "model2.pth"
torch.save(obj=model.state_dict(),f=model_path)

loaded_model_0 = TinyVGG()

# Load the state_dict of our saved model (this will update the new instance of our model with trained weights)
loaded_model_0.load_state_dict(torch.load(f=model_path))

# 1. Put the loaded model into evaluation mode
loaded_model_0.eval()

TinyVGG(
  (conv_block1): Sequential(
    (0): Conv2d(3, 25, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(25, 25, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block2): Sequential(
    (0): Conv2d(25, 25, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(25, 25, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=6400, out_features=3, bias=True)
  )
)